本章主要聚焦于聚类分析，它是一种无监督学习技术。其目的在于在数据中找到一个自然的分组，以便使同一簇(clustering)中的样本具有相同的模式，即更加相似。

#### 使用K-means方法按照相似度对对象进行分组

##### 使用sklearn进行k-means聚类

K-means算法属于基于原型的聚类方法(prototype-base clusting.) 本小节还将讨论另外两种聚类方法，`分层聚类和基于密度的聚类`。

* 其中，基于原型的聚类意味着每一个簇(cluster)都由一个原型(prototype)表示, 原型通常是具有连续特征的相似点的质心(centroid)(或者说均值)， 又或者是在类别型特征情况下的中心点(medoid)(最具有代表性的点或者是到当前簇内所有其他点距离最小化的点.) `这种聚类方法的缺点之一是必须人为指定聚类个数K`.

以下是一个使用K-means方法进行二维数据聚类的示例：